# Trees

To decide which column is the best for the current node we have 3 popular metrics;

    Gini Impurity.
    Information Gain.
    Variance reduction.
    
### Background
#### Entropy: 
    Entropy is a measure of the uncertainty or randomness of a system (In this case a feature, outcome of selecting a threshold for a feature etc.) 
    
    The Shannon entropy H(X) for a random variable X with possible outcomes x_1, x_2, ... x_n and corresponding probabilities p(x_1), p(x_2), ... p(x_n)  is defined as:

$$ H(X) = - \sum_{i=1}^{n} p(x_i) \log_2 p(x_i) $$

- #### Minimum Entropy
The minimum value of entropy is 0. This occurs when the system has no uncertainty, meaning there is only one possible outcome with a probability of 1.

- #### Maximum Entropy
The maximum value of entropy occurs when all outcomes are equally likely, meaning the probability distribution is uniform. For a system with 'n' possible outcomes, each with a probability of 1/n, the entropy is maximized. Mathematically, for p(x_i) = 1/n for all i.

$$ H(X) = - \sum_{i=1}^{n} {1/n}\log_2 1/n = \log_2 n $$


#### Gini Impurity
    Gini impurity is a measure used in decision tree algorithms to assess the quality of a split. It quantifies the likelihood of incorrect classification of a randomly chosen element if it were labeled according to the distribution of labels in the subset.
    
    The formula for Gini impurity G(X)G(X) for a dataset XX with possible classes c1,c2,…,cm and corresponding probabilities p(c1),p(c2),…,p(cm) is given by:

$$ G(X) = 1−\sum_{i=1}^m p(c_i)^2 $$

- #### Minimum Gini Impurity
The minimum value of Gini impurity is 0. This occurs when the dataset contains only one class (i.e., the distribution is perfectly pure). In this case, there is no uncertainty about the class of any randomly chosen element, and the Gini impurity is zero.

- #### Maximum Gini Impurity
The maximum value of Gini impurity occurs when the classes are equally distributed. For a binary classification (two classes), the maximum Gini impurity is 0.5. For n classes, it is 1-1/n.


#### Information gain
Information gain is a metric used to quantify the reduction in entropy or uncertainty about a random variable given additional information. It's commonly used in decision tree algorithms to decide which feature to split on at each step. Information gain is calculated as the difference between the entropy of the original dataset and the weighted sum of the entropies of the subsets after splitting on a particular feature.

Steps to calculate information gain:

   - Calculate the entropy of the dataset.
   - Calculate the entropy of each subset resulting from the split.
   - Compute the weighted sum of the entropies of the subsets.
   - Calculate the information gain as the difference between the original entropy and the weighted sum of the entropies of the subsets.

In [22]:
import math
import pandas as pd
from collections import Counter

def calculate_entropy(data: list):
    # Count the frequency of each item in the data
    counter = Counter(data)
    
    # Calculate the probabilities
    total_count = sum(counter.values())
    probabilities = [count / total_count for count in counter.values()]
    
    # Calculate the entropy
    entropy = -sum(p * math.log2(p) for p in probabilities)
    
    return entropy

# Example usage:
a1b2c3d4 = ['a', 'a', 'b', 'b', 'b', 'c', 'c', 'c', 'c']
a5d5 = ['a', 'a', 'a', 'a', 'a', 'a', 'b', 'b', 'b', 'b']


print(pd.Series(a1b2c3d4).value_counts())
entropy = calculate_entropy(a1b2c3d4)
print(f"Entropy of a1b2c3d4: {entropy}")

# print(pd.Series(a5d5).value_counts())
# entropy = calculate_entropy(a5d5)
# print(f"Entropy of a1b2c3d4: {entropy}")

c    4
b    3
a    2
Name: count, dtype: int64
Entropy of a1b2c3d4: 1.5304930567574826


In [21]:
def calculate_gini_impurity(data):
    # Count the frequency of each class in the data
    counter = Counter(data)
    
    # Calculate the probabilities
    total_count = sum(counter.values())
    probabilities = [count / total_count for count in counter.values()]
    
    # Calculate the Gini impurity
    gini_impurity = 1 - sum(p ** 2 for p in probabilities)
    
    return gini_impurity

# Example usage:
data = ['a', 'a', 'b', 'b', 'b', 'c', 'c', 'c', 'c']
gini_impurity = calculate_gini_impurity(data)
print(f"Gini Impurity: {gini_impurity}")


Gini Impurity: 0.6419753086419753


In [34]:
def calculate_information_gain(data: pd.DataFrame, feature:str, target:str):
    # Calculate the entropy of the original dataset
    print(data[target])
    total_entropy = calculate_entropy(data[target])
    print(total_entropy)
    # Calculate the values and counts for the split attribute
    counter = Counter(data[feature])
    total_count = sum(counter.values())
    print(total_count)
    
    
    weighted_entropy = 0
    for value, count in counter.items():
        subset = data[data[feature]==value][target]
        print(subset)
        subset_entropy = calculate_entropy(subset)
        weighted_entropy += (count / total_count) * subset_entropy
        
        # Calculate the information gain
    information_gain = total_entropy - weighted_entropy
    
    return information_gain

In [37]:
# Example usage:
data = [
    {'feature1': 'A', 'feature2': 'X', 'target': 'Yes'},
    {'feature1': 'A', 'feature2': 'Y', 'target': 'No'},
    {'feature1': 'B', 'feature2': 'X', 'target': 'Yes'},
    {'feature1': 'B', 'feature2': 'Y', 'target': 'No'},
    {'feature1': 'B', 'feature2': 'Y', 'target': 'Yes'},
]

data = pd.DataFrame(data)
split_attribute = 'feature2'
target_attribute = 'target'
info_gain = calculate_information_gain(data, split_attribute, target_attribute)
print(f"Information Gain for splitting on '{split_attribute}': {info_gain}")

0    Yes
1     No
2    Yes
3     No
4    Yes
Name: target, dtype: object
0.9709505944546686
5
0    Yes
2    Yes
Name: target, dtype: object
1     No
3     No
4    Yes
Name: target, dtype: object
Information Gain for splitting on 'feature2': 0.4199730940219749


In [51]:
import numpy as np
from collections import Counter

class DecisionTree:
    def __init__(self, criterion='gini', max_depth=None):
        self.criterion = criterion
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)
    
    def _build_tree(self, X, y, depth):
        # Stopping criteria
        if len(np.unique(y)) == 1 or len(y) == 0 or (self.max_depth and depth >= self.max_depth):
            return {'type': 'leaf', 'class': np.argmax(np.bincount(y))}
        
        # Calculate the best split
        best_feature, best_value = self._best_split(X, y)
        
        if best_feature is None:
            return {'type': 'leaf', 'class': np.argmax(np.bincount(y))}
        
        # Split the data
        if isinstance(best_value, (int, float)):
            left_indices = X[:, best_feature] <= best_value
            right_indices = X[:, best_feature] > best_value
        else:
            left_indices = X[:, best_feature] == best_value
            right_indices = X[:, best_feature] != best_value
            
        left_tree = self._build_tree(X[left_indices], y[left_indices], depth + 1)
        right_tree = self._build_tree(X[right_indices], y[right_indices], depth + 1)
        
        return {'type': 'node', 'feature': best_feature, 'value': best_value, 'left': left_tree, 'right': right_tree}

    def _best_split(self, X, y):
        best_feature = None
        best_value = None
        best_criterion_value = float('inf')
        
        n_samples, n_features = X.shape
        
        for feature in range(n_features):
            unique_values = np.unique(X[:, feature])
            for value in unique_values:
                if isinstance(value, (int, float)):
                    left_indices = X[:, feature] <= value
                    right_indices = X[:, feature] > value
                else:
                    left_indices = X[:, feature] == value
                    right_indices = X[:, feature] != value
                    
                if len(y[left_indices]) == 0 or len(y[right_indices]) == 0:
                    continue
                
                criterion_value = self._criterion(y[left_indices], y[right_indices])
                
                if criterion_value < best_criterion_value:
                    best_criterion_value = criterion_value
                    best_feature = feature
                    best_value = value
        
        return best_feature, best_value

    def _criterion(self, left_y, right_y):
        if self.criterion == 'gini':
            return self._gini_impurity(left_y, right_y)
        elif self.criterion == 'entropy':
            return self._information_gain(left_y, right_y)
        else:
            raise ValueError("Unsupported criterion. Use 'gini' or 'entropy'.")
    
    def _gini_impurity(self, left_y, right_y):
        n = len(left_y) + len(right_y)
        left_impurity = 1.0 - sum((np.sum(left_y == c) / len(left_y)) ** 2 for c in np.unique(left_y))
        right_impurity = 1.0 - sum((np.sum(right_y == c) / len(right_y)) ** 2 for c in np.unique(right_y))
        return (len(left_y) / n) * left_impurity + (len(right_y) / n) * right_impurity
    
    def _information_gain(self, left_y, right_y):
        n = len(left_y) + len(right_y)
        entropy_before = self._entropy(np.concatenate([left_y, right_y]))
        entropy_after = (len(left_y) / n) * self._entropy(left_y) + (len(right_y) / n) * self._entropy(right_y)
        return entropy_before - entropy_after
    
    def _entropy(self, y):
        probabilities = [np.sum(y == c) / len(y) for c in np.unique(y)]
        return -sum(p * np.log2(p) for p in probabilities)
    
    def predict(self, X):
        return [self._predict(inputs, self.tree) for inputs in X]
    
    def _predict(self, inputs, tree):
        if tree['type'] == 'leaf':
            return tree['class']
        
        feature_value = inputs[tree['feature']]
        if isinstance(tree['value'], (int, float)):
            branch = tree['left'] if feature_value <= tree['value'] else tree['right']
        else:
            branch = tree['left'] if feature_value == tree['value'] else tree['right']
        
        return self._predict(inputs, branch)

# Example usage:
X = np.array([
    ['A', 3], 
    ['A', 1], 
    ['B', 1], 
    ['B', 2], 
    ['B', 3], 
    ['B', 1]
])
y = np.array([0, 0, 1, 1, 1, 0])

tree = DecisionTree(criterion='entropy')
tree.fit(X, y)
predictions = tree.predict(X)
print(predictions)


[0, 0, 0, 1, 1, 0]


In [46]:
tree.tree

{'type': 'node',
 'feature': 0,
 'value': 'A',
 'left': {'type': 'leaf', 'class': 0},
 'right': {'type': 'node',
  'feature': 1,
  'value': '1',
  'left': {'type': 'leaf', 'class': 0},
  'right': {'type': 'leaf', 'class': 1}}}